# Final Project Check-in 2018-11-16

# Toxic Group

# Student Names

1. Xinran (Grace) Zhang
2. Shivee Singh
3. Neha Tevathia
4. Tianqi Wang
5. Meng-Ting(Joyce) Chang

## Overview

The project aims to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate, we will use a dataset of comments from Wikipedia’s talk page edits, collected by Kaggle. Improvements to the current model will hopefully help online discussion become more productive and respectful.

### Metrics

There are six labels, the results are evaluated on the mean column-wise ROC AUC. In other words, the score is the average of the individual AUCs of each predicted column. So the performance metrics for the model is AUC of each label.

##### Packages Load

In [1]:
# Packages for data processing and wrangling
import pandas as pd
import numpy as np
import re
import string

# Packages for visulization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py 
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Packages for feature engineering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from nltk.stem.wordnet import WordNetLemmatizer

# Packages for modeling
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Load Data

##### Data Load

In [2]:
train = pd.read_csv('./data/train.csv')
test_X = pd.read_csv('./data/test.csv')
test_y = pd.read_csv('./data/test_labels.csv')
submission = pd.read_csv('./data/sample_submission.csv')

##### Data Glimpse

Firstly we can take a look at training data.

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
train.shape

(159571, 8)

In training data, we have 159571 observations, for each observation, there is one unique id, and the predictor variable is the **comment_text**. The predicted variables are 6 labels that could show some toxic charateristics of the comment.

In [5]:
test_X.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [6]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
test_y.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [8]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [9]:
train.shape
test_X.shape

(153164, 2)

## Explanatory Data Analysis

We first check the predicted variable **Comment**.

In [10]:
train['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

Check the length distribution of the comment.

In [11]:
lens = train.comment_text.str.len()
lens.hist()

# Fit scikit-learn model

## Feature Engineering

In [12]:
def tokenize(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3.
    """
    #Convert everything to lowercase
    text=text.lower()
    # removed punctuation and tabs, new lines
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)
    words=nopunct.split(' ')
    
    words = [word.encode('ascii', 'ignore').decode('ascii') for word in words]
    
    #Lemmatize words
    lmtzr=WordNetLemmatizer()
    words=[lmtzr.lemmatize(w) for w in words]
    # drop words < 2 characters long
    words=[w for w in words if len(w)>2]
    return words

In [13]:
# Convert comments to term-document matrix using TFIDF
vectorizer = TfidfVectorizer(ngram_range=(1,1), analyzer= 'word' , tokenizer = tokenize , stop_words='english',
            strip_accents='unicode', use_idf=1)
X=vectorizer.fit_transform(train.comment_text)
X_test=vectorizer.transform(test_X.comment_text)
X.shape , X_test.shape

((159571, 159201), (153164, 159201))

In [24]:
vectorizer.get_feature_names()[-20:]

['zacatecas',
 'zach',
 'zachafinackus',
 'zacharia',
 'zacharias',
 'zachary',
 'zacherystaylor',
 'zaches',
 'zachinthebox',
 'zachlumi',
 'zachlumia',
 'zachlyda',
 'zack',
 'zackaback',
 'zackvision',
 'zacky',
 'zad',
 'zada',
 'zadar',
 'zadignose',
 'zadik',
 'zadock',
 'zadok',
 'zadora',
 'zadorov',
 'zaetta',
 'zafar',
 'zafarnama',
 'zafarnamah',
 'zafer',
 'zaffar',
 'zafirovna',
 'zag',
 'zagano',
 'zaghloul',
 'zagora',
 'zagorje',
 'zagreb',
 'zagroas',
 'zagurzem',
 'zagwe',
 'zahed',
 'zahedi',
 'zaheer',
 'zaheeruddin',
 'zahid',
 'zahir',
 'zahiri',
 'zahle',
 'zahler',
 'zahn',
 'zahra',
 'zaia',
 'zaidan',
 'zaidi',
 'zaido',
 'zaiger',
 'zaik',
 'zaim',
 'zain',
 'zainab',
 'zaira',
 'zaire',
 'zaista',
 'zaitut',
 'zaius',
 'zajdi',
 'zajednica',
 'zajjaj',
 'zajti',
 'zak',
 'zaka',
 'zakaria',
 'zakat',
 'zakawer',
 'zakhalesh',
 'zakharchenko',
 'zaki',
 'zakir',
 'zakk',
 'zakkenroller',
 'zakoni',
 'zaku',
 'zakusage',
 'zaky',
 'zakynthos',
 'zaladonis',
 '

## Modeling

Since there are six classes of predicted variables, we might need to create 6 classifiers, firstly we separate those labels.

In [25]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [30]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Evaluation Metric

In [36]:
# Logistic Regression Model AUC for all labels
lr_clf = LogisticRegression()
for label in labels:
    lr_clf.fit(X, train[label])
    predicted_prob = lr_clf.predict(X_test)
    lr_auc = roc_auc_score(test_y[test_y[label]!=-1][label], predicted_prob[test_y[label]!=-1])
    print(f"The AUC Score of Logistic Regression Model for {label} is {round(lr_auc,4)}")

The AUC Score of Logistic Regression Model for toxic is 0.8305
The AUC Score of Logistic Regression Model for severe_toxic is 0.6552
The AUC Score of Logistic Regression Model for obscene is 0.8
The AUC Score of Logistic Regression Model for threat is 0.5945
The AUC Score of Logistic Regression Model for insult is 0.7568
The AUC Score of Logistic Regression Model for identity_hate is 0.6223


Then we try SVM 

In [24]:
# Build the model
svm_clf = SVC(kernel = 'linear')

# Fit the model
svm_clf.fit(train_feature, train['toxic'])

# Predict
svm_predicted_toxic = svm_clf.predict(test_feature)

# Check the model performance
svm_auc = roc_auc_score(test_y[test_y['toxic']!=-1].toxic,
                        svm_predicted_toxic[test_y['toxic']!=-1])

print(f"The AUC Score of SVM Model is {round(svm_auc,4)}")

The AUC Score of SVM Model is 0.8253


In [29]:
print(f"The AUC Score of SVM Model is {round(svm_auc,4)}")

The AUC Score of SVM Model is 0.8661


In [30]:
from sklearn.metrics import recall_score
recall_score(test_y[test_y['toxic']!=-1].toxic, predicted_toxic[test_y['toxic']!=-1])

0.6886699507389162

### Reference

https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline
